<a href="https://colab.research.google.com/github/hhmida/ESB/blob/main/Application_SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://hhmida.github.io/tp-hadoop-iset/assets/credit_risk_dataset.csv

--2022-12-02 09:07:56--  https://hhmida.github.io/tp-hadoop-iset/assets/credit_risk_dataset.csv
Resolving hhmida.github.io (hhmida.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to hhmida.github.io (hhmida.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1804682 (1.7M) [text/csv]
Saving to: ‘credit_risk_dataset.csv’

credit_risk_dataset 100%[===================>]   1.72M  --.-KB/s    in 0.008s  

2022-12-02 09:07:56 (228 MB/s) - ‘credit_risk_dataset.csv’ saved [1804682/1804682]



In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 53 kB/s 
     |████████████████████████████████| 199 kB 66.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a3d32269970a174a6cd1319e144653752e11e6e9950352a17075ff8d6beab5b4
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Exercice SparkSQL').getOrCreate()

In [4]:
credits = spark.read.csv("credit_risk_dataset.csv", header=True).cache()

In [6]:
credits.printSchema()

root
 |-- person_age: string (nullable = true)
 |-- person_income: string (nullable = true)
 |-- person_home_ownership: string (nullable = true)
 |-- person_emp_length: string (nullable = true)
 |-- loan_intent: string (nullable = true)
 |-- loan_grade: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- loan_int_rate: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- loan_percent_income: string (nullable = true)
 |-- cb_person_default_on_file: string (nullable = true)
 |-- cb_person_cred_hist_length: string (nullable = true)



In [7]:
credits.createOrReplaceTempView('credits')

In [9]:
spark.sql("select sum(loan_amnt), avg(person_emp_length) from credits where loan_status=1").show()

+--------------+----------------------+
|sum(loan_amnt)|avg(person_emp_length)|
+--------------+----------------------+
|   7.7125375E7|     4.137562261939642|
+--------------+----------------------+



In [15]:
credits.where((credits.loan_status==1)).agg({'loan_amnt':'sum', 'person_emp_length':'mean'}).show()

+----------------------+--------------+
|avg(person_emp_length)|sum(loan_amnt)|
+----------------------+--------------+
|     4.137562261939642|   7.7125375E7|
+----------------------+--------------+



In [18]:
spark.sql('select nbr/total *100, loan_status from (select count(*) as nbr, loan_status from credits group by loan_status), (select count(*) as total from credits)').show()

+---------------------+-----------+
|((nbr / total) * 100)|loan_status|
+---------------------+-----------+
|    78.18360394094718|          0|
|   21.816396059052824|          1|
+---------------------+-----------+



In [19]:
total = spark.sql("select count(*) as total from credits")
nbr = spark.sql("select count(*) as nbr, loan_status from credits group by loan_status")


In [29]:
total.join(nbr).createOrReplaceTempView('temp')
spark.sql("select nbr/total*100, loan_status from temp").show()

+---------------------+-----------+
|((nbr / total) * 100)|loan_status|
+---------------------+-----------+
|    78.18360394094718|          0|
|   21.816396059052824|          1|
+---------------------+-----------+

